In [1]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
pwd

'/home/linux/Documents/bigdata2020/data/notabook/Inhwan'

In [79]:
from selenium.webdriver.chrome.options import Options

opts = Options()
# opts.add_argument("--headless")
opts.add_argument("--no-sandbox")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36")


In [77]:
driver = webdriver.Chrome('./chromedriver', options=opts)
df = pd.read_csv('/home/linux/Documents/bigdata2020/data/tour_data.csv')
df1 = df.loc[:,['tour_id','title']]

In [80]:
title_list = []
tour_id_list = []
overview_list = []

In [82]:
def main():
    global driver    
    
    driver.get('https://map.kakao.com/')
    
    driver.implicitly_wait(3)
    driver.delete_all_cookies()
    for i in range(0, 10000): # 범위 설정 **** 변경 ****
        driver.implicitly_wait(10)
        print('####', i+1)
        search(df1['title'][i], i+1)
    driver.quit()
    print("finish")

In [73]:
#카카오맵 첫화면에서 관광지명으로 검색한뒤 첫번째 결과로 crawilng 함수 호출하는 함수
def search(place, tour_id):
    global driver
    
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)
    html = driver.page_source
    
    try :
        soup = BeautifulSoup(html, 'html.parser')
        
        place = soup.select('.placelist > .PlaceItem')[0] # 검색된 장소 목록
        try :
            sleep(1)
            crawling(place, tour_id)
        
        except ElementNotInteractableException:
            print('not found')
        finally:
            search_area.clear()
    except NoSuchElementException:
        print('no search result')
        search_area.clear()
    except IndexError:
        print('no search result')
        search_area.clear()

In [50]:
def crawling(place, tour_id):
    place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
    detail_page_xpath = '//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]'
    driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
    sleep(1)
    
    print('####', place_name, tour_id)
    
    # 첫 페이지
    extract_overview(place_name, tour_id)
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [51]:
def extract_overview(place_name, tour_id):
    global driver
    
    ret = True
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 개요 찾기
    overview = soup.select('.txt_introduce')
    
    # 개요가 있는 경우
    if len(overview) != 0 :
        title_list.append(place_name)
        tour_id_list.append(tour_id)
        overview_list.append(overview[0].text)
        print(tour_id, overview)
    
    else :
        print('no overreview in extract')
        title_list.append(place_name)
        tour_id_list.append(tour_id)
        overview = None
        overview_list.append(overview)
        ret = False
    return ret

In [83]:
if __name__ == "__main__":
    driver = webdriver.Chrome('./chromedriver', options=opts)
    main()

#### 1
#### 코스테이 1
no overreview in extract
#### 2
no search result
#### 3
#### 가나아트센터 3
no overreview in extract
#### 4


WebDriverException: Message: chrome not reachable
  (Session info: chrome=86.0.4240.111)


In [99]:
pd.DataFrame({'tour_id' :tour_id_list, 'overview' : overview_list})

,tour_id,overview
0,221,None
1,222,\n 세계적으로 유명한 현대미술 작가의 작품과 그...
2,223,\n 1700년의 역사를 지닌 한국불교의 전통과 ...
3,225,None
4,226,None
5,227,None
6,228,\n 헌정기념관은 1998년 5월 29일 국회개원...
7,229,"\n 여성가족위원회,국회사무처,통일외교통상위원회,..."
8,230,None
